In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-140158")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,quniform
import os

# Specify parameter sampler
parameter_space={
    'C':uniform(0.5, 2),
    'max_iter':quniform(50, 150, 10)}
ps =  RandomParameterSampling(parameter_space, properties=None) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=None, slack_amount=0.1, delay_evaluation=0) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

source_directory='./training'
path='Users/odl_user_140158/train.py'

# Create a SKLearn estimator for use with train.py
est =SKLearn(source_directory, compute_target=cpu_cluster, vm_size='STANDARD_D2_V2', vm_priority=None, 
             entry_script=path, script_params={'C':None,'max_iter':None}, use_docker=True, custom_docker_image=None, 
             image_registry_details=None, user_managed=False, conda_packages=None, pip_packages=None, 
             conda_dependencies_file_path=None, pip_requirements_file_path=None, conda_dependencies_file=None, 
             pip_requirements_file=None, environment_variables=None, environment_definition=None, inputs=None, 
             shm_size=None, resume_from=None, max_run_duration_seconds=None, framework_version=None,
             _enable_optimized_mode=False, _disable_validation=True, _show_lint_warnings=False,
             _show_package_warnings=False) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=100, max_concurrent_runs=None, max_duration_minutes=10080, 
                                     policy=policy, estimator=est, run_config=None, resume_from=None,
                                     resume_child_runs=None, pipeline=None)### YOUR CODE HERE ###

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run=exp.submit(hyperdrive_config, tags=None, **kwargs)
RunDetails(run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run=run.get_best_run_by_primary_metric
best_run, hyperdrive_model = best_run.get_output()
joblib.dump(hyperdrive_model, 'hyperdrive_model.joblib')

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path, validate=True, include_path=False, infer_column_types=True, 
                                                set_column_types=None, separator=',', 
                                                header=True, partition_format=None, support_multi_line=False, 
                                                empty_as_string=False, encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=32)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=x_train,
    label_column_name=y_train,
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.
best_run, automl_model = run.get_output()
joblib.dump(automl_model, 'automl_model.joblib')
### YOUR CODE HERE ###